In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
df1 = pd.read_csv("yjmob100k-dataset1.csv")
homes = pd.read_csv("homes.csv")
workplaces = pd.read_csv("workplaces.csv")

grid = gpd.read_file("grid_bl_2449.geojson")
weather_data = pd.read_csv("weather_data_15mins.csv")
processed_weather_data = pd.read_csv("processed_weather_data.csv")

total_rg = pd.read_csv("total_rg.csv")
daily_rg = pd.read_csv("daily_rg.csv")

In [ ]:
# adathalmaz lecsökkentése az első 10000 felhasználóra

df_test = df1[df1['uid'].between(0, 9999)]

In [ ]:
# date oszlop felvétele 
base_date = pd.to_datetime("2019-09-15")
df_test['date'] = base_date + pd.to_timedelta(df_test['d'], unit='D')

# days of week oszlop felvétele
days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
df_test['days_of_week'] = df_test['d'].apply(lambda x: days_of_week[x % 7])

# cell_id oszlop felvétele
df_test["cell_id"] = df_test["x"].astype(str) + "_" + df_test["y"].astype(str)


# lat és lon oszlopok felvétele
# Centroidok számítása új változóba
grid_centroids = grid.copy()
grid_centroids["geometry"] = grid_centroids.centroid
# WGS84-re (lat/lon) transzformálás
grid_centroids_wgs84 = grid_centroids.to_crs(epsg=4326)
# lat/lon oszlopok
grid_centroids_wgs84["lon"] = grid_centroids_wgs84.geometry.x
grid_centroids_wgs84["lat"] = grid_centroids_wgs84.geometry.y

#grid_centroids_wgs84.head()

# merge a df_test-el a cell_id alapján
df_test = df_test.merge(
    grid_centroids_wgs84[["x", "y", "lat", "lon"]],
    on=["x", "y"],
    how="left"
)


In [ ]:
# OPEN-METEO WEATHER DATA HOZZÁADÁSA

# minden második sor törlése a weather_data DataFrame-ből (csak egész órás és fél órás adatok kellenek 15 percesek helyett)
weather_data = weather_data.iloc[::2].reset_index(drop=True)

# time átalakitása d és t oszlopokhoz
weather_data['t'] = weather_data['time'].astype(str)

# Szétválasztás a 'T' mentén: első rész a dátum, második az idő
weather_data[['t','h']] = weather_data['t'].str.split('T', expand=True)

# time átállitása 1-48-ra
def time_to_halfhour_slot(tstr):
    hour, minute = map(int, tstr.split(':')[:2])
    return hour * 2 + (1 if minute == 30 else 0) + 1

weather_data['h'] = weather_data['h'].apply(time_to_halfhour_slot)

# weather_code mapping
# Egyszerusitett mapping az ML modellezéshez
weather_group_map = {
    0: "Clear",
    1: "Clear",
    2: "Clear",
    3: "Cloudy",
    45: "Fog", 48: "Fog",
    51: "Rain", 53: "Rain", 55: "Rain",
    56: "Rain", 57: "Rain",
    61: "Rain", 63: "Rain", 65: "Rain",
    66: "Rain", 67: "Rain",
    71: "Snow", 73: "Snow", 75: "Snow",
    77: "Snow",
    80: "Rain", 81: "Rain", 82: "Rain",
    85: "Snow", 86: "Snow",
    95: "Thunderstorm",
    96: "Thunderstorm", 99: "Thunderstorm"
}

weather_data["weather_code (wmo code)"] = weather_data["weather_code (wmo code)"].map(weather_group_map)

# oszlopok átnevezése
weather_data = weather_data.rename(columns={
    't': 'date',
    'h': 't',
    'weather_code (wmo code)': 'weather_code',
    'rain (mm)' : 'rain',
    'temperature_2m (°C)' : 'temperature'
})

# time oszlop törlése
weather_data = weather_data.drop(columns=['time'])

# weather_data mentése csv fájlba
weather_data.to_csv("processed_weather_data.csv", index=False)

In [ ]:
# PROCESSED WEATHER DATA BEOLVASÁSA
processed_weather_data = pd.read_csv("processed_weather_data.csv")

# df_test és idojarasi adatok mergelése
# A merge előtt alakítsd mindkét 'date' oszlopot datetime típusra!
df_test['date'] = pd.to_datetime(df_test['date'])
processed_weather_data['date'] = pd.to_datetime(processed_weather_data['date'])

df_test = pd.merge(df_test, processed_weather_data, on=['date', 't'], how='left')

In [ ]:
# RADIUS OF GYRATION HOZZÁADÁSA

# total_rog mergelése
df_test = df_test.merge(total_rg, on='uid', how='left', suffixes=('', '_total_rg'))

# daily_rog mergelése
# df_test date oszlop -> datetime64[ns] (idő nélkül)
df_test["date"] = pd.to_datetime(df_test["date"]).dt.normalize()

# daily_rg date oszlop -> datetime64[ns]
daily_rg["date"] = pd.to_datetime(daily_rg["date"]).dt.normalize()

# merge
df_test = df_test.merge(
    daily_rg,
    on=["uid", "date"],
    how="left",
    suffixes=("", "_daily_rg")
)

In [ ]:
# workplaces + homes HOZZÁADÁSA

# workplaces hozzáadása a df_test-hez
df_test["is_home"] = df_test.merge(
    homes.assign(is_home=1),
    on=["uid", "x", "y"],
    how="left"
)["is_home"].fillna(0).astype(int)

# is_workplace oszlop hozzáadása a df_test-hez
df_test["is_workplace"] = df_test.merge(
    workplaces.assign(is_workplace=1),
    on=["uid", "x", "y"],
    how="left"
)["is_workplace"].fillna(0).astype(int)

In [ ]:
# fraction_missing hozzáadása  -> hiányzó adatok aránya az adott userre nézve
def fraction_missing(df_test, n_days=75, slots_per_day=48):
    expected = n_days * slots_per_day
    return (
        df_test.groupby("uid").size()
          .reset_index(name="observed")
          .assign(fraction_missing=lambda g: 1 - g["observed"]/expected)
    )

fraction_df = fraction_missing(df_test, n_days=75, slots_per_day=48)

df_test = df_test.merge(fraction_df[["uid", "fraction_missing"]], on="uid", how="left")

In [ ]:
# unique_cells hozzáadása  -> hány különböző cellában fordult elő az adott user
unique_cells = (
    df_test.groupby("uid")["cell_id"]
           .nunique()
           .reset_index(name="unique_cells_count")
)

df_test = df_test.merge(unique_cells, on="uid", how="left")

In [ ]:
# df_test mentése csv fájlba
df_test.to_csv("df_test_10k.csv", index=False)